In [ ]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2020산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

In [1]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [2]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/본문_형태소분석/2020산림복지_본문_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

323


In [3]:
df = pd.read_csv("../dataset/본문_형태소분석/2020산림복지_본문_형태소분석(형태소만).csv", encoding='UTF-8')
df

,split_str,org_idx,correct_str,morphs
0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0,이제는 열심히 부지런히 최선을 잘 봉사하는 삶을 내려놓아도 잘 산다고 해서 대충 살...,"['이제', '는', '열심히', '부지런히', '최선', '을', '잘', '봉사..."
1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0,필리핀 단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지...,"['필리핀', '단', '기', '선교', '를', '시작', '으로', '2020..."
2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0,교육 수료증 5개 임산물 가공 산야초 산림경영 회복적 교사 심리학 국가자격증 1개 ...,"['교육', '수료증', '5', '개', '임산물', '가공', '산야', '초'..."
3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0,뭐든 열심히 하는 나에게 맞는일맞는 일더위에도 재미있었고 사람을 멀리하는 나에게 관...,"['뭐', '든', '열심히', '하는', '나', '에게', '맞는', '일', ..."
4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0,오지랖이 넓어 실수도 많이 있었지! 교육청에서 좋은 분들도 만났고 과장님께서는 늘 ...,"['오지랖', '이', '넓어', '실수', '도', '많이', '있었지', '!'..."
...,...,...,...,...
164988,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다,5787,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다.,"['더불어', '치병', '중인', '블로그', '이웃', '님', '들과도', '..."
164989,다음은 2020년 1월 7일부터 20일까지의 일기다,5787,다음은 2020년 1월 7일부터 20일까지의 일기다,"['다음', '은', '2020년', '1월', '7일', '부터', '20일', ..."
164990,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 냥이 ...,5787,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 고양이...,"['올해', '열', '여섯', '살이', '된', '반려', '묘', '냥', '..."
164991,2020년 1월 7일 하루 종일 비가 내린다,5787,2020년 1월 7일 하루 종일 비가 내린다.,"['2020년', '1월', '7일', '하루', '종일', '비', '가', '내..."


In [4]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,correct_str,morphs,joined_morphs
0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0,이제는 열심히 부지런히 최선을 잘 봉사하는 삶을 내려놓아도 잘 산다고 해서 대충 살...,"['이제', '는', '열심히', '부지런히', '최선', '을', '잘', '봉사...",이제 는 열심히 부지런히 최선 을 잘 봉사 하는 삶 을 내려놓아도 잘 산다 고 해서...
1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0,필리핀 단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지...,"['필리핀', '단', '기', '선교', '를', '시작', '으로', '2020...",필리핀 단 기 선교 를 시작 으로 2020년 코로나 여파 에도 나 의 삶은 여전히 ...
2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0,교육 수료증 5개 임산물 가공 산야초 산림경영 회복적 교사 심리학 국가자격증 1개 ...,"['교육', '수료증', '5', '개', '임산물', '가공', '산야', '초'...",교육 수료증 5 개 임산물 가공 산야 초 산림 경영 회복 적 교사 심리학 국가 자격...
3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0,뭐든 열심히 하는 나에게 맞는일맞는 일더위에도 재미있었고 사람을 멀리하는 나에게 관...,"['뭐', '든', '열심히', '하는', '나', '에게', '맞는', '일', ...",뭐 든 열심히 하는 나 에게 맞는 일 맞는 일 더위 에도 재미있었고 사람 을 멀리 ...
4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0,오지랖이 넓어 실수도 많이 있었지! 교육청에서 좋은 분들도 만났고 과장님께서는 늘 ...,"['오지랖', '이', '넓어', '실수', '도', '많이', '있었지', '!'...",오지랖 이 넓어 실수 도 많이 있었지 ! 교육청 에서 좋은 분 들 도 만났고 과장 ...
...,...,...,...,...,...
164988,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다,5787,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다.,"['더불어', '치병', '중인', '블로그', '이웃', '님', '들과도', '...",더불어 치병 중인 블로그 이웃 님 들과도 파이팅 을 함께 외쳐 본다 .
164989,다음은 2020년 1월 7일부터 20일까지의 일기다,5787,다음은 2020년 1월 7일부터 20일까지의 일기다,"['다음', '은', '2020년', '1월', '7일', '부터', '20일', ...",다음 은 2020년 1월 7일 부터 20일 까지의 일기 다
164990,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 냥이 ...,5787,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 고양이...,"['올해', '열', '여섯', '살이', '된', '반려', '묘', '냥', '...",올해 열 여섯 살이 된 반려 묘 냥 이 하루 종일 토 하던 날 기운 없이 앉아 있는...
164991,2020년 1월 7일 하루 종일 비가 내린다,5787,2020년 1월 7일 하루 종일 비가 내린다.,"['2020년', '1월', '7일', '하루', '종일', '비', '가', '내...",2020년 1월 7일 하루 종일 비 가 내 린다 .


In [5]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,correct_str,morphs,joined_morphs,joined_sentences
0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0,이제는 열심히 부지런히 최선을 잘 봉사하는 삶을 내려놓아도 잘 산다고 해서 대충 살...,"['이제', '는', '열심히', '부지런히', '최선', '을', '잘', '봉사...",이제 는 열심히 부지런히 최선 을 잘 봉사 하는 삶 을 내려놓아도 잘 산다 고 해서...,이제 는 열심히 부지런히 최선 을 잘 봉사 하는 삶 을 내려놓아도 잘 산다 고 해서...
1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0,필리핀 단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지...,"['필리핀', '단', '기', '선교', '를', '시작', '으로', '2020...",필리핀 단 기 선교 를 시작 으로 2020년 코로나 여파 에도 나 의 삶은 여전히 ...,이제 는 열심히 부지런히 최선 을 잘 봉사 하는 삶 을 내려놓아도 잘 산다 고 해서...
2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0,교육 수료증 5개 임산물 가공 산야초 산림경영 회복적 교사 심리학 국가자격증 1개 ...,"['교육', '수료증', '5', '개', '임산물', '가공', '산야', '초'...",교육 수료증 5 개 임산물 가공 산야 초 산림 경영 회복 적 교사 심리학 국가 자격...,이제 는 열심히 부지런히 최선 을 잘 봉사 하는 삶 을 내려놓아도 잘 산다 고 해서...
3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0,뭐든 열심히 하는 나에게 맞는일맞는 일더위에도 재미있었고 사람을 멀리하는 나에게 관...,"['뭐', '든', '열심히', '하는', '나', '에게', '맞는', '일', ...",뭐 든 열심히 하는 나 에게 맞는 일 맞는 일 더위 에도 재미있었고 사람 을 멀리 ...,이제 는 열심히 부지런히 최선 을 잘 봉사 하는 삶 을 내려놓아도 잘 산다 고 해서...
4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0,오지랖이 넓어 실수도 많이 있었지! 교육청에서 좋은 분들도 만났고 과장님께서는 늘 ...,"['오지랖', '이', '넓어', '실수', '도', '많이', '있었지', '!'...",오지랖 이 넓어 실수 도 많이 있었지 ! 교육청 에서 좋은 분 들 도 만났고 과장 ...,이제 는 열심히 부지런히 최선 을 잘 봉사 하는 삶 을 내려놓아도 잘 산다 고 해서...
...,...,...,...,...,...,...
164988,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다,5787,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다.,"['더불어', '치병', '중인', '블로그', '이웃', '님', '들과도', '...",더불어 치병 중인 블로그 이웃 님 들과도 파이팅 을 함께 외쳐 본다 .,올해 들어 호흡 이 고르지 못 해 걷기 운동 을 매일 6 km 에서 5 km 로 줄...
164989,다음은 2020년 1월 7일부터 20일까지의 일기다,5787,다음은 2020년 1월 7일부터 20일까지의 일기다,"['다음', '은', '2020년', '1월', '7일', '부터', '20일', ...",다음 은 2020년 1월 7일 부터 20일 까지의 일기 다,올해 들어 호흡 이 고르지 못 해 걷기 운동 을 매일 6 km 에서 5 km 로 줄...
164990,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 냥이 ...,5787,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 고양이...,"['올해', '열', '여섯', '살이', '된', '반려', '묘', '냥', '...",올해 열 여섯 살이 된 반려 묘 냥 이 하루 종일 토 하던 날 기운 없이 앉아 있는...,올해 들어 호흡 이 고르지 못 해 걷기 운동 을 매일 6 km 에서 5 km 로 줄...
164991,2020년 1월 7일 하루 종일 비가 내린다,5787,2020년 1월 7일 하루 종일 비가 내린다.,"['2020년', '1월', '7일', '하루', '종일', '비', '가', '내...",2020년 1월 7일 하루 종일 비 가 내 린다 .,올해 들어 호흡 이 고르지 못 해 걷기 운동 을 매일 6 km 에서 5 km 로 줄...


In [6]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,이제 는 열심히 부지런히 최선 을 잘 봉사 하는 삶 을 내려놓아도 잘 산다 고 해서...
1,1,플라스틱 쓰레기 문제 육식 을 위 한 가축 산업 으로 인한 문제 산림 파괴 석유 누...
2,2,석 가 공자 예수 는 성인 이자 현인 이기도 하다 이분 들 은 중생 을 구제 하는 ...
3,3,2020년 통영 거제 환경운동 연합 선정 올해 의 환경 뉴스 통영 거제 환경운동 연...
4,4,"블로그 전남 7 , 530 개 의 글 전남 목록 열기 전남도 산림 행정 종합 평가 ..."
...,...,...
5783,5783,신종 코로나바이러스 2019 NCO 피해 기업 에 자금 지원 계획 2020년 2월 ...
5784,5784,"대구시 의원 동구 3 동구 4 민주당 권오준 최완 식 vs 한국 당 3 , 4 명 ..."
5785,5785,한 달 에 한 달 에 한 번 씩 진행 되는 그루 매니저 역량 강화 교육 2월 교육 ...
5786,5786,경기 수원 공무원 학원 에듀윌 수원 학원 2020 서울시 채용 공고 안내 2020 ...


In [7]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [8]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [9]:
tdm.shape

(5788, 148644)

In [10]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.0108977, 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

In [11]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)

In [12]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [13]:
tfidf_DF

,0,00,000,0000,0001,000100,0001002,0001003,0001003308,0001004238,...,힘주어,힘줄,힘차게,힘차고,힘차다,힘찬,힘찹니다,힙,힙니,힙합
0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.010898,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.043168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.012218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,0.032617,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5784,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5785,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5786,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# 이제 다 완성된 TF-IDF 행렬은 txt로 저장해줌.
tfidf_DF.to_csv("../dataset/본문_TF-IDF/2020산림복지_본문_TF-IDF행렬.txt", index=False, encoding='UTF-8')

KeyboardInterrupt: 

In [50]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,51.032181
1,00,50.503403
2,000,19.454386
3,0001,0.049739
4,0001076717,0.034615
...,...,...
134284,힙니,0.062174
134285,힙스터,0.058074
134286,힙합,0.405733
134287,힝,0.086247


In [51]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/본문_TF-IDF/2020산림복지_본문_TF-IDF합.txt", index=False, encoding='UTF-8')